# Assignment 3: Web scraping
A simple scraper to extract some data  data for 50 “Samand” cars, manufactured after 1385 from the site: https://bama.ir

Group Members:


*   Ali Zahedzadeh
*   Melika Noubakhtian



**Import Libraries**

We use ```request``` to get html source code of page

We use ```selenium``` , because bama webpage is INFITE SCROLING so we cant get all webpage content (selenium help us)

We use ```BeautifulSoup``` , it help us to extract value of each html tag that we need

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
import time

```get_links_for_cars``` function :

This function is using ```selenium``` for execute a script that open a browser (like a bot , and we use Firefox) then scroll down webpage until reach the limit

At end the function return a list that contain 50 links about detail information for each samand car that manufactured after 1385

In [2]:
def get_links_for_cars(base_url, car_name, start_year):
    final_url = base_url + car_name + '?' + 'year' + '=' + start_year
    options = webdriver.FirefoxOptions()
    driver = webdriver.Firefox(options=options)
    driver.get(final_url)

    items = []

    last_height = driver.execute_script('return document.body.scrollHeight')
    itemTargetCount = 50

    while itemTargetCount > len(items):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(2)
        new_height = driver.execute_script('return document.body.scrollHeight')

        if new_height == last_height:
            break

        last_height = new_height

        elements = driver.find_elements(By.CSS_SELECTOR, 'div.bama-ad-holder')

        href_elements = []
        for element in elements:
            html_data = element.get_attribute("outerHTML")
            soup = BeautifulSoup(html_data)
            href_value = soup.find('a')['href']
            final_value = 'https://bama.ir' + href_value
            href_elements.append(final_value)
        items = href_elements


    if len(items) < 50:
        clean_links = items
    else:
        clean_links = items[0:50]

    return clean_links

```get_clean_page_text``` function :

This function use ```requests``` library for get html source of each url

In [3]:
def get_clean_page_text(url, header):
    response = requests.get(url, headers=header)
    return response.content

```crawler``` function :
This function iterate each url that comes from ```get_links_for_cars``` function and then use ```BeautifulSoup``` for extract each data we need.

For each car ad we saved all data (that we need) in a python dictionary.

We scrap this data from each ad (And we null any value that is not mentioned in the advertisement):

*   Price
*   Year
*   Title
*   Description
*   Mileage
*   Fuel_type
*   Transmition_type
*   Body_status
*   Body_color
*   Inside_color

At end this function return a list of dictionaries that contain above data

In [4]:
def crawler(all_urls, headers):
    key_value_map = {}
    all_cars_clean_data = []
    for url in all_urls:  
        #extract html code for each page
        new_content = get_clean_page_text(url, headers)
        soup = BeautifulSoup(new_content, 'html.parser')

        #extract price , year, title and descripton for each car
        try:
            price = float(soup.find('span',class_='bama-ad-detail-price__price').text.strip().replace(',',''))
        except:
            price = np.nan
        key_value_map['Price'] = price

        try:
            year = int(soup.find('span',class_='bama-ad-detail-title__subtitle').text.strip())
        except:
            year = np.nan
        key_value_map['Year'] = year

        try:
            title = soup.find('h1',class_='bama-ad-detail-title__title').text.strip()
        except:
            title = np.nan
        key_value_map['Title'] = title
        
        try:
            description = soup.find('div',class_='desc').text.strip()
        except:
            description = np.nan
        key_value_map['Description'] = description

        print(price, year, title)
        print(description)

        '''
        0 -> mileage
        1 -> fuel_type
        2 -> transmition_type
        3 -> body_status
        4 -> body_color
        5 -> inside_color
        '''

        other_data = soup.find_all('div',class_='bama-vehicle-detail-with-icon__detail-holder')
        
        #extract mileage value (clean value)
        try:
            mileage = int(other_data[0].text.split()[1].replace(',',''))
        except:
            mileage = np.nan
        print(mileage)
        key_value_map['Mileage'] = mileage

        #extract fuel_type value (clean value)
        try:
            text = other_data[1].text.strip()
            fuel_type = text[text.startswith('نوع سوخت') and len(' نوع سوخت'):]
        except:
            fuel_type = np.nan
        print(fuel_type)
        key_value_map['Fuel_type'] = fuel_type

        #extract transmition_type value (clean value)
        try:
            text = other_data[2].text.strip()
            transmition_type = text[text.startswith('گیربکس ') and len('گیربکس '):]
        except:
            transmition_type = np.nan
        print(transmition_type)
        key_value_map['Transmition_type'] = transmition_type

        #extract body_status value (clean value)
        try:
            text = other_data[3].text.strip()
            body_status = text[text.startswith('وضعیت بدنه ') and len('وضعیت بدنه '):]
        except:
            body_status = np.nan
        print(body_status)
        key_value_map['Body_status'] = body_status

        #extract body_color value (clean value)
        try:
            text = other_data[4].text.strip()
            body_color = text[text.startswith('رنگ بدنه ') and len('رنگ بدنه '):]
        except:
            body_color = np.nan
        print(body_color)
        key_value_map['Body_color'] = body_color

        #extract inside_color value (clean value)
        try:
            text = other_data[5].text.strip()
            inside_color = text[text.startswith('رنگ داخلی ') and len('رنگ داخلی '):]
        except:
            inside_color = np.nan
        print(inside_color)
        key_value_map['Inside_color'] = inside_color
        all_cars_clean_data.append(key_value_map)
        key_value_map = dict()
    
    return all_cars_clean_data


Now we set parameters and call functions

Our ```crawler``` function also displays the log for each ad that scraped

In [5]:
Headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0'}
base_url = 'https://bama.ir/car/'
car_name = 'samand'
start_year = '1386'

clean_links = get_links_for_cars(base_url, car_name, start_year)
final_data = crawler(clean_links, Headers)

383000000.0 1400 سمند، LX
nan
nan
بنزینی
دنده ای
بدون رنگ
سفید
داخل کرم
230000000.0 1390 سمند، LX
nan
400000
بنزینی
دنده ای
چند لکه رنگ
نقره ای
داخل کرم
579000000.0 1402 سمند، سورن
فول شرکتی رینگ آلومینیومی مانیتور فابریک و دوربین دنده عقب کروز کنترل  صندلی برقی
nan
بنزینی
دنده ای
بدون رنگ
سفید
داخل مشکی
325000000.0 1394 سمند، LX
از سال 94دست خودم بوذه تازه موتور و جلوبندی و دیسک و تسمه تایم و لوازم مصرفی عوض شده. درب راننده رو بخاطر فرورفتگی عوض کردم. خط و خش هم دارد. قیمت هم با خریدار راه میام با تیبا و ساینا و کوییک صفر هم معاوضه دارم بازدید هم اسلام اباد غرب
217000
دوگانه سوز
دنده ای
یک لکه رنگ
سفید
داخل کرم
550000000.0 1401 سمند، سورن
با رینگ،یکسال بیمه
nan
بنزینی
دنده ای
بدون رنگ
سفید
داخل مشکی
nan 1395 سمند، سورن
روکش صندلی  دودی هدلایت منبع اگزوز HKS همه روشه ماشین از پشت ضربه خورده با مشتری واقعی کنار میایم با تشکر
160000
بنزینی
دنده ای
یک لکه رنگ
سفید
داخل خاکستری
380000000.0 1397 سمند، LX
ماشین فوق‌العاده سالم تماس سرویس ها به موقع  انجام شده
240000
بنزینی
دنده ای
بدون رنگ
س

Now we save clean data in a pandas dataframe

<div dir=rtl style="direction: rtl;text-align: right;line-height:200%;font-family:vazir;font-size:medium">
<font face="vazir" size=3>
<center>

| ستون | توضیح |
|:-----------:|:------:|
| Title |عنوان ماشین|
| Year | سال تولید|
| Fuel_type |نوع سوخت (بنزینی ، گاز سوز ، دو گانه سوز)|
| Transmition_type | گیربکس (دنده ای ، اتومات)|
| Body_color | رنگ بدنه|
| Body_status | وضعیت بدنه|
| Inside_color |رنگ داخلی|
| Mileage | کارکرد|
| Description | توضیحات آگهی|
| Price | قیمت|

</center>
</font>
</div>

In [6]:
df = pd.DataFrame(final_data)
#reorder columns of dataframe
df = df[['Title', 'Year', 'Fuel_type', 'Transmition_type', 'Body_color', 'Body_status', 'Inside_color', 'Mileage', 'Description', 'Price']]
df

,Title,Year,Fuel_type,Transmition_type,Body_color,Body_status,Inside_color,Mileage,Description,Price
0,سمند، LX,1400,بنزینی,دنده ای,سفید,بدون رنگ,داخل کرم,NaN,NaN,383000000.0
1,سمند، LX,1390,بنزینی,دنده ای,نقره ای,چند لکه رنگ,داخل کرم,400000.0,NaN,230000000.0
2,سمند، سورن,1402,بنزینی,دنده ای,سفید,بدون رنگ,داخل مشکی,NaN,فول شرکتی رینگ آلومینیومی مانیتور فابریک و دور...,579000000.0
3,سمند، LX,1394,دوگانه سوز,دنده ای,سفید,یک لکه رنگ,داخل کرم,217000.0,از سال 94دست خودم بوذه تازه موتور و جلوبندی و ...,325000000.0
4,سمند، سورن,1401,بنزینی,دنده ای,سفید,بدون رنگ,داخل مشکی,NaN,با رینگ،یکسال بیمه,550000000.0
5,سمند، سورن,1395,بنزینی,دنده ای,سفید,یک لکه رنگ,داخل خاکستری,160000.0,روکش صندلی دودی هدلایت منبع اگزوز HKS همه روش...,NaN
6,سمند، LX,1397,بنزینی,دنده ای,سفید,بدون رنگ,داخل قهوه ای,240000.0,ماشین فوق‌العاده سالم تماس سرویس ها به موقع ا...,380000000.0
7,سمند، LX,1397,بنزینی,دنده ای,سفید,بدون رنگ,داخل نوک مدادی,112000.0,NaN,395000000.0
8,سمند، LX,1400,بنزینی,دنده ای,سفید,بدون رنگ,داخل کرم,46000.0,ماشین صحیح وسالم ، فوری فروشی ، زیر قیمت .,385000000.0
9,سمند، LX,1397,بنزینی,دنده ای,خاکستری,بدون رنگ,داخل کرم,180000.0,از صفر دست خودم بوده رینگ آلومینیومی صلیبی بدو...,390000000.0


Now we save pandas dataframe as a Excel file

In [7]:
# set file name
file_name = 'bama-cars-samand.xlsx'
# saving the excel
df.to_excel(file_name)

![saved_file_pic](final.png)